In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
import pandas as pd
import seaborn as sns
import tensorflow as tf
#tfd = tf.contrib.distributions
import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors
layers = tf.contrib.layers
tf.set_random_seed(0)
from hepmc.core.densities.qcd import ee_qq_ng, export_hepmc, import_hepmc
from hepmc.core.densities.nice import Nice
from hepmc.core.phase_space.rambo import RamboOnDiet
from hepmc.core.phase_space.mapping import MappedDensity
from hepmc.core.sampling import Sample
from hepmc.core.integration.importance import ImportanceMC
from hepmc.core.sampling import Sample, AcceptRejectSampler
#from hepmc.core.markov.metropolis import DefaultMetropolis
#from hepmc.core.proposals import Gaussian

In [ ]:
sherpa_weighted_sample = import_hepmc('../samples/qcd/2-3/sherpa_weighted.hepmc')
sherpa_weighted_sample = Sample(data=rambo_mapping.map_inverse(sherpa_weighted_sample.data), weights=sherpa_weighted_sample.weights)

In [ ]:
data = pd.DataFrame(sherpa_weighted_sample.data)
f = sns.PairGrid(data)
f = f.map_diag(plt.hist, bins=15)
f = f.map_offdiag(plt.hist2d, bins=15)

In [ ]:
sns.distplot(sherpa_weighted_sample.weights, kde=False)

In [ ]:
sherpa_weighted_sample.weights.mean()/sherpa_weighted_sample.weights.max()

In [ ]:
eeqqg = ee_qq_ng(1, 100., 5., .3)
rambo_mapping = RamboOnDiet(100., 3)
mapped = MappedDensity(eeqqg, rambo_mapping)
training_sample = import_hepmc('../samples/qcd/2-3/training.hepmc')
training_sample = Sample(data=rambo_mapping.map_inverse(training_sample.data), weights=training_sample.weights)

In [ ]:
data = pd.DataFrame(training_sample.data)
f = sns.PairGrid(data)
f = f.map_diag(plt.hist, bins=15)
f = f.map_offdiag(plt.hist2d, bins=15)

In [ ]:
%time nice = Nice(training_sample, train_iters=2000)

In [ ]:
importance_sampler = ImportanceMC(mapped, nice)
%time nice_sample = importance_sampler.sample(100000)

In [ ]:
sns.distplot(nice_sample.weights[nice_sample.weights < 5000], kde=False)

In [ ]:
mapped_sample = Sample(data=rambo_mapping.map(nice_sample.data), weights=nice_sample.weights)
export_hepmc(100., mapped_sample, "../samples/qcd/2-3/realnvp.hepmc")

In [ ]:
bound = nice_sample.weights.max()

In [ ]:
nice_sample.weights.mean()/2500

In [ ]:
bound=2500
print('M:', bound)
sample = importance_sampler.sample(1)
x = sample.data
print('x:', x)
weight = sample.weights
print('weight:', weight)
rand = np.random.rand(1)
print('u:', rand)
sampling_pdf = nice.sess.run(nice.dist.prob(x.astype(np.float32)))
print('g(x):', sampling_pdf)
target_pdf = mapped.pdf(x)
print('f(x):', target_pdf)
print('f(x)/(M*g(x)):', target_pdf/(bound*sampling_pdf))
print('weight/M:', weight/bound)

In [ ]:
sampler = AcceptRejectSampler(target=mapped, bound=bound, sampler=importance_sampler, sampling_pdf=nice.pdf)

In [ ]:
%time sample = sampler.sample(10000, expected_efficiency=.07)

In [ ]:
class AcceptRejectSampler(object):
    """ Acceptance Rejection method for sampling a given pdf.
    
    The method uses a known distribution and sampling method to propose
    samples which are then accepted with the probability
    pdf(x)/(c * sampling_pdf(x)), thus producing the desired distribution. 
    The resulting sample is unweighted.
    
    .. todo::
        Handle points that lie above the bound.
    """

    def __init__(self, target, bound: float, 
            sampler = None, sampling_pdf = None) -> None:
        """
        Parameters
        ----------
        target
            Unnormalized desired probability distribution of the sample.
        bound
            Constant such that pdf(x) <= bound * sampling_pdf(x)
            for all x in the range of sampling.
        sampler
            The sampler which generates the sample. The default is a uniform sampler.
        """
        self.target = target
        self.bound = bound
        self.ndim = target.ndim

        if sampler is None:
            sampler = UniformSampler(target)
            def sampling_pdf(x):
                return np.ones(x.size)

        self.sampler = sampler
        self.sampling_pdf = sampling_pdf

    def sample(self, sample_size: int, expected_efficiency: float = 1.) -> None:
        x = np.empty((sample_size, self.ndim))

        #indices = np.arange(sample_size)
        n_todo = sample_size
        trials = 0
        while n_todo > 0:
            print('n_todo:', n_todo)
            trials += int(n_todo/expected_efficiency)
            sample = self.sampler.sample(int(n_todo/expected_efficiency))
            proposal = sample.data
            #accept = np.random.rand(indices.size) * self.bound * self.sampling_pdf(proposal) <= self.target.pdf(proposal)
            #accept = np.random.rand(indices.size) * self.bound <= sample.weights
            #accept = np.random.rand(indices.size) < sample.weights / self.bound
            u = np.random.rand(int(n_todo/expected_efficiency))
            accept = u < sample.weights / self.bound
            n_accept = accept.sum()
            if n_accept <= n_todo:
                x[sample_size-n_todo:sample_size-n_todo+n_accept] = proposal[accept]
            else:
                accepted = proposal[accept]
                x[sample_size-n_todo:] = accepted[:n_todo]
            n_todo -= n_accept
            #x[indices[accept]] = proposal[accept]
            #indices = indices[np.logical_not(accept)]
        print('Unweighting eff.:', sample_size/trials)
        return Sample(data=x, target=self.target)

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
DTYPE=tf.float32
NP_DTYPE=np.float32
USE_BATCHNORM = False

In [ ]:
batch_size = 100
num_bijectors = 5
train_iters = 1e4

In [ ]:
eeqqg = ee_qq_ng(1, 100., 5., .3)
rambo_mapping = RamboOnDiet(100., 3)
mapped = MappedDensity(eeqqg, rambo_mapping)
#met = DefaultMetropolis(mapped, Gaussian(5, .01))
#start = met.sample(5000, np.random.rand(5)).data[-1] # warmup
#%time met_sample = met.sample(10000, start)
training_sample = import_hepmc('../samples/qcd/2-3/training.hepmc')
X = rambo_mapping.map_inverse(training_sample.data)
#X = np.hstack((X, np.ones((X.shape[0], 1)))) # make ndim even

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(X.astype(NP_DTYPE))
dataset = dataset.repeat()
dataset = dataset.shuffle(buffer_size=X.shape[0])
dataset = dataset.prefetch(3 * batch_size)
dataset = dataset.batch(batch_size)
data_iterator = dataset.make_one_shot_iterator()
x_samples = data_iterator.get_next()

In [ ]:
class BatchNorm(tfb.Bijector):
    def __init__(self, eps=1e-5, decay=0.95, validate_args=False, name="batch_norm"):
        super(BatchNorm, self).__init__(
            event_ndims=1, validate_args=validate_args, name=name)
        self._vars_created = False
        self.eps = eps
        self.decay = decay

    def _create_vars(self, x):
        n = x.get_shape().as_list()[1]
        with tf.variable_scope(self.name):
            self.beta = tf.get_variable('beta', [1, n], dtype=DTYPE)
            self.gamma = tf.get_variable('gamma', [1, n], dtype=DTYPE)
            self.train_m = tf.get_variable(
                'mean', [1, n], dtype=DTYPE, trainable=False)
            self.train_v = tf.get_variable(
                'var', [1, n], dtype=DTYPE, initializer=tf.ones_initializer, trainable=False)
        self._vars_created = True

    def _forward(self, u):
        if not self._vars_created:
            self._create_vars(u)
        return (u - self.beta) * tf.exp(-self.gamma) * tf.sqrt(self.train_v + self.eps) + self.train_m

    def _inverse(self, x):
        # Eq 22. Called during training of a normalizing flow.
        if not self._vars_created:
            self._create_vars(x)
        # statistics of current minibatch
        m, v = tf.nn.moments(x, axes=[0], keep_dims=True)
        # update train statistics via exponential moving average
        update_train_m = tf.assign_sub(
            self.train_m, self.decay * (self.train_m - m))
        update_train_v = tf.assign_sub(
            self.train_v, self.decay * (self.train_v - v))
        # normalize using current minibatch statistics, followed by BN scale and shift
        with tf.control_dependencies([update_train_m, update_train_v]):
            return (x - m) * 1. / tf.sqrt(v + self.eps) * tf.exp(self.gamma) + self.beta

    def _inverse_log_det_jacobian(self, x):
        # at training time, the log_det_jacobian is computed from statistics of the
        # current minibatch.
        if not self._vars_created:
            self._create_vars(x)
        _, v = tf.nn.moments(x, axes=[0], keep_dims=True)
        abs_log_det_J_inv = tf.reduce_sum(
            self.gamma - .5 * tf.log(v + self.eps))
        return abs_log_det_J_inv

In [ ]:
base_dist = tfd.MultivariateNormalDiag(loc=tf.zeros([5], DTYPE))
#uniform = tfd.Uniform(low=tf.zeros([5], DTYPE), high=tf.ones([5], DTYPE))
#base_dist = tfd.Independent(
#    distribution=tfd.Uniform(low=tf.zeros([5], DTYPE), high=tf.ones([5], DTYPE)), 
#    reinterpreted_batch_ndims=1)
#base_dist = tfd.Independent(
#    distribution=tfd.TruncatedNormal(loc=tf.zeros([5], DTYPE), scale=1., low=tf.zeros([5], DTYPE), high=tf.ones([5], DTYPE)),
#    reinterpreted_batch_ndims=1)

In [ ]:
uniform.batch_shape

In [ ]:
base_dist.event_shape

In [ ]:
base_dist.batch_shape

In [ ]:
base_dist.sample(2).eval()

In [ ]:
base_dist.prob(base_dist.sample(10)).eval()

In [ ]:
bijectors = []

for i in range(num_bijectors):
    #bijectors.append(NVPCoupling(D=2, d=1, layer_id=i))
    bijectors.append(tfb.RealNVP(num_masked=3, shift_and_log_scale_fn=tfb.real_nvp_default_template(hidden_layers=[512, 512])))
    if USE_BATCHNORM and i % 2 == 0:
        # BatchNorm helps to stabilize deep normalizing flows, esp. Real-NVP
        bijectors.append(BatchNorm(name='batch_norm%d' % i))
    #if i % 2 == 0:
    #    bijectors.append(tfb.Permute(permutation=[2, 3, 4, 0, 1]))
    #else:
    #    bijectors.append(tfb.Permute(permutation=[3, 4, 0, 1, 2]))
    #bijectors.append(tfb.Permute(permutation=[3, 4, 2, 0, 1]))
    bijectors.append(tfb.Permute(permutation=[2, 3, 4, 0, 1]))
# Discard the last Permute layer.
flow_bijector = tfb.Chain(list(reversed(bijectors[:-1])))

In [ ]:
dist = tfd.TransformedDistribution(
    distribution=base_dist,
    bijector=flow_bijector)

In [ ]:
# visualization
x = base_dist.sample(8000)
samples = [x]
names = [base_dist.name]
for bijector in reversed(dist.bijector.bijectors):
    x = bijector.forward(x)
    samples.append(x)
    names.append(bijector.name)

In [ ]:
sess.run(tf.global_variables_initializer())

In [ ]:
results = sess.run(samples)
f, arr = plt.subplots(1, len(results), figsize=(4 * (len(results)), 4))
X0 = results[0]
for i in range(len(results)):
    X1 = results[i]
    idx = np.logical_and(X0[:, 0] < 0, X0[:, 1] < 0)
    arr[i].scatter(X1[idx, 0], X1[idx, 1], s=10, color='red')
    idx = np.logical_and(X0[:, 0] > 0, X0[:, 1] < 0)
    arr[i].scatter(X1[idx, 0], X1[idx, 1], s=10, color='green')
    idx = np.logical_and(X0[:, 0] < 0, X0[:, 1] > 0)
    arr[i].scatter(X1[idx, 0], X1[idx, 1], s=10, color='blue')
    idx = np.logical_and(X0[:, 0] > 0, X0[:, 1] > 0)
    arr[i].scatter(X1[idx, 0], X1[idx, 1], s=10, color='black')
    arr[i].set_xlim([-10, 10])
    arr[i].set_ylim([-10, 10])
    arr[i].set_title(names[i])

In [ ]:
loss = -tf.reduce_mean(dist.log_prob(x_samples))
train_op = tf.train.AdamOptimizer(1e-4).minimize(loss)

In [ ]:
sess.run(tf.global_variables_initializer())

In [ ]:
%%time
NUM_STEPS = int(train_iters)
global_step = []
np_losses = []
for i in range(NUM_STEPS):
    _, np_loss = sess.run([train_op, loss])
    if i % 1000 == 0:
        global_step.append(i)
        np_losses.append(np_loss)
    #if i % int(1e4) == 0:
        print(i, np_loss)
start = 0
plt.plot(np_losses[start:])

In [ ]:
# visualization
x = base_dist.sample(8000)
samples = [x]
names = [base_dist.name]
for bijector in reversed(dist.bijector.bijectors):
    x = bijector.forward(x)
    samples.append(x)
    names.append(bijector.name)

In [ ]:
results = sess.run(samples)
#X0 = results[0]
#rows = 2
#cols = int(len(results) / 2)
#f, arr = plt.subplots(2, cols, figsize=(4 * (cols), 4 * rows))
#i = 0
#for r in range(rows):
#    for c in range(cols):
#        X1 = results[i]
#        idx = np.logical_and(X0[:, 0] < 0, X0[:, 1] < 0)
#        arr[r, c].scatter(X1[idx, 0], X1[idx, 1], s=10, color='red')
#        idx = np.logical_and(X0[:, 0] > 0, X0[:, 1] < 0)
#        arr[r, c].scatter(X1[idx, 0], X1[idx, 1], s=10, color='green')
#        idx = np.logical_and(X0[:, 0] < 0, X0[:, 1] > 0)
#        arr[r, c].scatter(X1[idx, 0], X1[idx, 1], s=10, color='blue')
#        idx = np.logical_and(X0[:, 0] > 0, X0[:, 1] > 0)
#        arr[r, c].scatter(X1[idx, 0], X1[idx, 1], s=10, color='black')
#        arr[r, c].set_xlim([-5, 5])
#        arr[r, c].set_ylim([-5, 5])
#        arr[r, c].set_title(names[i])
#
#        i += 1

In [ ]:
data = pd.DataFrame(results[0])
f = sns.PairGrid(data)
f = f.map_diag(plt.hist, bins=15)
f = f.map_offdiag(plt.hist2d, bins=15)

In [ ]:
data = pd.DataFrame(results[1])
f = sns.PairGrid(data)
f = f.map_diag(plt.hist, bins=15)
f = f.map_offdiag(plt.hist2d, bins=15)

In [ ]:
data = pd.DataFrame(results[2])
f = sns.PairGrid(data)
f = f.map_diag(plt.hist, bins=15)
f = f.map_offdiag(plt.hist2d, bins=15)

In [ ]:
data = pd.DataFrame(results[3])
f = sns.PairGrid(data)
f = f.map_diag(plt.hist, bins=15)
f = f.map_offdiag(plt.hist2d, bins=15)

In [ ]:
data = pd.DataFrame(results[4])
f = sns.PairGrid(data)
f = f.map_diag(plt.hist, bins=15)
f = f.map_offdiag(plt.hist2d, bins=15)

In [ ]:
data = pd.DataFrame(results[5])
f = sns.PairGrid(data)
f = f.map_diag(plt.hist, bins=15)
f = f.map_offdiag(plt.hist2d, bins=15)

In [ ]:
data = pd.DataFrame(results[6])
f = sns.PairGrid(data)
f = f.map_diag(plt.hist, bins=15)
f = f.map_offdiag(plt.hist2d, bins=15)

In [ ]:
data = pd.DataFrame(results[7])
f = sns.PairGrid(data)
f = f.map_diag(plt.hist, bins=15)
f = f.map_offdiag(plt.hist2d, bins=15)

In [ ]:
data = pd.DataFrame(results[8])
f = sns.PairGrid(data)
f = f.map_diag(plt.hist, bins=15)
f = f.map_offdiag(plt.hist2d, bins=15)

In [ ]:
data = pd.DataFrame(results[9])
f = sns.PairGrid(data)
f = f.map_diag(plt.hist, bins=15)
f = f.map_offdiag(plt.hist2d, bins=15, range=[[0, 1], [0, 1]])

In [ ]:
data = pd.DataFrame(results[10])
f = sns.PairGrid(data)
f = f.map_diag(plt.hist, bins=15)
f = f.map_offdiag(plt.hist2d, bins=15)

In [ ]:
data = pd.DataFrame(results[11])
f = sns.PairGrid(data)
f = f.map_diag(plt.hist, bins=15)
f = f.map_offdiag(plt.hist2d, bins=15)

In [ ]:
data = pd.DataFrame(results[12])
f = sns.PairGrid(data)
f = f.map_diag(plt.hist, bins=15)
f = f.map_offdiag(plt.hist2d, bins=15)

In [ ]:
data = pd.DataFrame(results[13])
f = sns.PairGrid(data)
f = f.map_diag(plt.hist, bins=15)
f = f.map_offdiag(plt.hist2d, bins=15)

In [ ]:
data = pd.DataFrame(results[14])
f = sns.PairGrid(data)
f = f.map_diag(plt.hist, bins=15)
f = f.map_offdiag(plt.hist2d, bins=15)

In [ ]:
data = pd.DataFrame(results[15])
f = sns.PairGrid(data)
f = f.map_diag(plt.hist, bins=15)
f = f.map_offdiag(plt.hist2d, bins=15, range=[[0, 1], [0, 1]])

In [ ]:
def integrate(target, eval_count):
    xs = np.empty((eval_count, 5), dtype=np.float32)
    ys = np.empty(eval_count, dtype=np.float32)
    weights = np.empty(eval_count)
    trials = 0

    indices = np.arange(eval_count)
    while indices.size > 0:
        trials += indices.size
        x = base_dist.sample(indices.size)
        samples = [x]
        for bijector in reversed(dist.bijector.bijectors):
            x = bijector.forward(x)
            samples.append(x)
        results = sess.run(samples)
        x = results[-1]
        #x = self.dist.rvs(indices.size)
        y = target.pdf(x)
        in_bounds = y != 0.
        xs[indices[in_bounds]] = x[in_bounds]
        ys[indices[in_bounds]] = y[in_bounds]
        indices = indices[np.logical_not(in_bounds)]

    #weights = ys / self.dist.pdf(xs)
    weights = ys / dist.prob(xs).eval()
    integral = eval_count/trials * weights.mean()
    stderr = np.sqrt(weights.var() * eval_count/trials**2)
    sample = Sample(data=xs, target=target, pdf=ys, weights=weights)

    return sample, integral, stderr

In [ ]:
sample, integral, err = integrate(mapped, 100000)

In [ ]:
mapped_sample = Sample(data=rambo_mapping.map(sample.data), weights=sample.weights)
export_hepmc(100., mapped_sample, "../samples/qcd/2-3/realnvp.hepmc")

In [ ]:
bound = sample.weights.max()

In [ ]:
sampler = AcceptRejectSampler(target=eeqqg, bound=bound, sampler=importance_sampler, sampling_pdf=sarge.pdf)